<a href="https://colab.research.google.com/github/TakeruShimada/tigerfish/blob/master/notebooks/dl/dl_No1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#論理ゲートの実装

##AND

In [21]:
import numpy as np

In [22]:
# ANDゲート
def AND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.7
    tmp = np.sum(w * x) + b
    if tmp <= 0:
        return 0
    else:
        return 1

In [23]:
# ANDゲートの出力確認
print(AND(0, 0))
print(AND(1, 0))
print(AND(0, 1))
print(AND(1, 1))

0
0
0
1


##NAND

In [24]:
# NANDゲート
def NAND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([-0.5, -0.5])
    b = 0.7
    tmp = np.sum(w * x) + b
    if tmp <= 0:
        return 0
    else:
        return 1

##OR

In [25]:
# ORゲート
def OR(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.2
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1

##XOR

In [26]:
# XORゲート
def XOR(x1, x2):
    s1 = NAND(x1, x2)
    s2 = OR(x1, x2)
    y = AND(s1, s2)
    return y

#IrisDatasetを使ってみる

##ライブラリのインポート

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

##データセットのロード

In [28]:
# データをロード
iris = load_iris()
# 訓練用データとテスト用データを分割
x_train, x_test, y_train, y_test = train_test_split(iris["data"], iris["target"])

# 訓練用データ
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)  # pytorchの仕様で出力の型はint64にする

# テスト用データ
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)  # pytorchの仕様で出力の型はint64にする

##Datasetの作成

In [29]:
# 下記の３種類を用いることでdatasetを作成することができる
# 基本はこの３種類を使うことでdatasetを作成
class Iris_Dataset(Dataset):
    def __init__(self, x=None, y=None):
        self.data = x
        self.targets = y

    # 長さを返す
    def __len__(self):
        return len(self.targets)

    # batchの値を操作
    def __getitem__(self, item):
        x = self.data[item]
        y = self.targets[item]
        return x, y

In [30]:
# 訓練データ
train_ds = Iris_Dataset(x_train, y_train)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
# テストデータ
test_ds = Iris_Dataset(x_test, y_test)
test_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

##ネットワークの実装

In [31]:
class Model(nn.Module):
    def __init__(self, in_fitures: int, mid_fitures: int, n_class: int) -> None:
        super(Model, self).__init__()
        self.fc: nn.Module = nn.Sequential(
            nn.Linear(in_fitures, mid_fitures),
            nn.ReLU(),  # 活性化関数(ReLU関数)
            nn.Linear(mid_fitures, mid_fitures)
        )
        self.clf = nn.Linear(mid_fitures, n_class)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc(x)
        x = torch.relu_(x)
        x = self.clf(x)
        return x


model = Model(4, 12, 3)
print(model)
optimizer = optim.Adam(model.parameters())  # 最適化アルゴリズム
criterion = nn.CrossEntropyLoss()  # 損失関数

for epoch in tqdm(range(100)):
    for batch in train_loader:
        a, b = batch
        optimizer.zero_grad()
        out = model(a)
        loss = criterion(out, b)
        loss.backward()
        optimizer.step()
print(loss.detach().item())

Model(
  (fc): Sequential(
    (0): Linear(in_features=4, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=12, bias=True)
  )
  (clf): Linear(in_features=12, out_features=3, bias=True)
)



0.19601209461688995


##ネットワークの精度

In [32]:
result = model(x_test)
predicted = torch.max(result, 1)[1]
print("{:.2f}".format(sum(p == t for p, t in zip(predicted, y_test)) / len(x_test)))

1.00
